# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [122]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [130]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kahului,20.8947,-156.4700,28.01,71,40,5.14,US,1970-01-01 00:00:01.682124339
1,1,Hobart,-42.8794,147.3294,18.92,65,75,8.75,AU,1970-01-01 00:00:01.682124069
2,2,Mataura,-46.1927,168.8643,15.71,70,94,1.65,NZ,1970-01-01 00:00:01.682124340
3,3,Avera,33.1940,-82.5271,21.97,58,100,2.95,US,1970-01-01 00:00:01.682124340
4,4,Rikitea,-23.1203,-134.9692,26.05,81,100,7.78,PF,1970-01-01 00:00:01.682124311


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [124]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)
# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [125]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.dropna(how = 'any')
#city_data_df.head()

#selct cities with temperatures less than 20'c and greater than 28'c
city_temp_df = city_data_df[(city_data_df["Temperature"]<=28) & (city_data_df["Temperature"]>=20)]
#city_data_temp_df.max()

#select cities with cloud cover less than 20%
city_temp_cloud_df = city_temp_df[(city_temp_df["Cloudiness"]<=0)]
# city_temp_cloud_df.count()
# city_temp_cloud_df.max()

#select cities with wind speed less than 4.5
city_weather_df = city_temp_cloud_df[(city_temp_cloud_df["Wind Speed"]<4.5)]

city_weather_df.count()

City_ID        14
City           14
Latitude       14
Longitude      14
Temperature    14
Humidity       14
Cloudiness     14
Wind Speed     14
Country        14
Date           14
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [126]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_df.copy(deep=True)
#hotel_df.head()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head()

,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,Kapaa,22.0752,-159.3190,27.99,79,0,3.60,US,1970-01-01 00:00:01.682124351,
1,83,San Angelo,31.4638,-100.4370,27.12,11,0,4.12,US,1970-01-01 00:00:01.682124397,
2,117,Linguere,15.3953,-15.1193,26.12,57,0,4.12,SN,1970-01-01 00:00:01.682124416,
3,127,Tosham,28.8833,75.9167,23.77,28,0,2.56,IN,1970-01-01 00:00:01.682124423,
4,155,Luxor,25.6989,32.6421,22.03,20,0,1.54,EG,1970-01-01 00:00:01.682124244,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [127]:

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame and find te nearest hotel
for index, row in hotel_df.iterrows():
    #define the longitude and latitude for each search
    #add filters and bias parameters with the current city's latitude and longitude 
    categories = "accommodation.hotel"
    Latitude = row['Latitude']
    Longitude = row['Longitude']
    radius = 20000
    filters = f'circle:{Longitude},{Latitude},{radius}'
    bias = f'proximity:{Longitude},{Latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Define the parameters dictionary
    params = {
    "categories":categories,
    "filters":filters,
    "bias":bias,
    "apiKey":geoapify_key 
    }
    
    # Make an API request using the params dictionary and convert API response to json format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kapaa - nearest hotel: Pono Kai Resort
San Angelo - nearest hotel: Hilton Hotel
Linguere - nearest hotel: Thiossane Plaza
Tosham - nearest hotel: OYO Hotel Raza
Luxor - nearest hotel: Salam
Wichita Falls - nearest hotel: Regency Inn & Suites
Iberia - nearest hotel: Inn of Iberia
Saint-Pierre - nearest hotel: Tropic Hotel
Salinas - nearest hotel: The Traveler's Hotel
Saint-Paul - nearest hotel: Hôtel d'application
Khanpur - nearest hotel: Lavish Hotel
Da Nang - nearest hotel: Thanh Thanh Hotel
Santa Rosalia - nearest hotel: Sol y Mar
Chabahar - nearest hotel: هتل سپیده


,City_ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,Kapaa,22.0752,-159.3190,27.99,79,0,3.60,US,1970-01-01 00:00:01.682124351,Pono Kai Resort
1,83,San Angelo,31.4638,-100.4370,27.12,11,0,4.12,US,1970-01-01 00:00:01.682124397,Hilton Hotel
2,117,Linguere,15.3953,-15.1193,26.12,57,0,4.12,SN,1970-01-01 00:00:01.682124416,Thiossane Plaza
3,127,Tosham,28.8833,75.9167,23.77,28,0,2.56,IN,1970-01-01 00:00:01.682124423,OYO Hotel Raza
4,155,Luxor,25.6989,32.6421,22.03,20,0,1.54,EG,1970-01-01 00:00:01.682124244,Salam
5,167,Wichita Falls,33.9137,-98.4934,22.08,14,0,3.09,US,1970-01-01 00:00:01.682124447,Regency Inn & Suites
6,317,Iberia,29.9502,-91.7507,24.04,68,0,3.09,US,1970-01-01 00:00:01.682124533,Inn of Iberia
7,354,Saint-Pierre,-21.3393,55.4781,24.82,69,0,2.57,RE,1970-01-01 00:00:01.682124557,Tropic Hotel
8,386,Salinas,36.6777,-121.6555,25.45,59,0,4.12,US,1970-01-01 00:00:01.682124580,The Traveler's Hotel
9,430,Saint-Paul,-21.0096,55.2707,25.06,75,0,1.87,RE,1970-01-01 00:00:01.682124605,Hôtel d'application


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [128]:
%%capture --no-display


map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    hover_cols = ['Hotel Name', 'Country'],
    frame_width = 700,
    frame_height = 500,
    size = 100,
    scale = 0.8,
    color = "City"
)

# Display the map plot
map_plot_2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Hotel Name,Country)